In [ ]:
pip install PyPDF2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 220 kB 5.1 MB/s 


In [ ]:
import requests,sys,io
import PyPDF2
from urllib.parse import quote
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#from tika import parser
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)


Mounted at /content/drive


In [ ]:

headers = {
    'Accept': 'application/json, text/javascript, */*; q=0.01',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
    'Origin': 'https://www.mhc.tn.gov.in',
    'Referer': 'https://www.mhc.tn.gov.in/judis/madras-do/index.php/casestatus/datewise',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
    'sec-ch-ua': '"Google Chrome";v="107", "Chromium";v="107", "Not=A?Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

data = {
    'dtfrom': '2022-09-01',
    'dtto': '2022-10-01',
}

response = requests.post(
    'https://www.mhc.tn.gov.in/judis/madras-do/index.php/casestatus/findtext',
    headers=headers,
    data=data,
    verify=False
)

In [ ]:
def threadable(pdf_url,file_path):
  output=''
  data = requests.get(pdf_url,verify=False).content
  #string_parsed = parser.from_buffer(data)
  with io.BytesIO(data) as open_pdf_file:
     read_pdf = PyPDF2.PdfFileReader(open_pdf_file)
     for i in range(read_pdf.numPages):
        page = read_pdf.getPage(i)
        output += page.extractText()
  #string_parsed = parser.from_buffer(data)
  #if string_parsed['content'] is not None:
    #if 'auction' in string_parsed['content']:
  if 'auction' in output:
    with open(file_path, 'wb') as f:
      f.write(data)

In [ ]:
path = "/content/drive/MyDrive/highcourt/"
futures=[]

with ProcessPoolExecutor(max_workers=20) as executor:
  with tqdm(total=len(response.json()["find_data"]),file=sys.stdout) as progress:
    for item in response.json()["find_data"]:
      pdf_name=quote(item["do_pdf_name"])
      pdf_url='https://www.mhc.tn.gov.in/judis/madras-do/index.php/casestatus/viewpdf/' + pdf_name
      file_path=path + str(item["casetype"]) + "_" +  str(item["caseno"]) + "_" + str(item["caseyr"]) + "_" + str(item["jud_dt"]) + ".pdf"
      future =executor.submit(threadable,pdf_url,file_path)
      future.add_done_callback(lambda p: progress.update())
      futures.append(future)

    results= [future.result() for future in tqdm(futures)]


  0%|          | 0/12332 [00:00<?, ?it/s]


  0%|          | 0/12332 [00:00<?, ?it/s]

  0%|          | 1/12332 [00:02<6:53:02,  2.01s/it]


  0%|          | 1/12332 [00:00<2:31:18,  1.36it/s]

  0%|          | 12/12332 [00:04<39:57,  5.14it/s]

...

  0%|          | 34/12332 [00:07<19:22, 10.58it/s]


  0%|          | 22/12332 [00:05<42:54,  4.78it/s]

  1%|          | 115/12332 [00:15<22:29,  9.06it/s]


  0%|          | 35/12332 [00:14<1:32:23,  2.22it/s]

  1%|          | 116/12332 [00:15<26:58,  7.55it/s]
